<a href="https://colab.research.google.com/github/VasylDvorakDS/Probability_and_statistics_theory/blob/master/Probability_theory_Home_Work_Lesson_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задача**

Провести дисперсионный анализ для определения того, есть ли различия среднего роста среди взрослых футболистов, хоккеистов и штангистов. Даны значения роста в трех группах случайно выбранных спортсменов: Футболисты: 173, 175, 180, 178, 177, 185, 183, 182. Хоккеисты: 177, 179, 180, 188, 177, 172, 171, 184, 180. Штангисты: 172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170.

Выбираем уровень статистической значимости alpha=0.05.

Для дисперсионного анализа сначала проверим условия применимости:
1.	Значения групп должны следовать нормальному распределению. Проверим с помощью теста Шапиро - Уилка
2.	Однородность дисперсий. Проверим с помощью теста Барлет

3.	Независимость измерений. Измерения производились для разных людей

Желательно чтобы длинна выборок для каждой группы была одинаковой.



In [ ]:
import numpy as np
from scipy import stats

In [ ]:
football_players=np.array([173, 175, 180, 178, 177, 185, 183, 182])
hockey_players=np.array([177, 179, 180, 188, 177, 172, 171, 184, 180])
weightlifters=np.array([172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170])

alpha=0.05

print(len(football_players), len(hockey_players), len(weightlifters))

8 9 11


Напишем функцию Шапиро-Уилка для определения нормальности

In [ ]:
def test_shapiro (input):
  _,p_value=stats.shapiro(input)
  if p_value > alpha:
    print('Выборка из нормального распределения')
  else:
    print('Выборка из ненормального распределения')


In [ ]:
test_shapiro(football_players)

Выборка из нормального распределения


In [ ]:
test_shapiro(hockey_players)

Выборка из нормального распределения


In [ ]:
test_shapiro(weightlifters)

Выборка из нормального распределения


Все выборки из нормального распределения. Первое условие выполняется.
Проведём теста Барлетта.

In [ ]:
_,p_value = stats.bartlett(football_players, hockey_players, weightlifters)
p_value

0.7929254656083131

p_value>alpha, 0.79>0.05. Различий в дисперсиях для трёх выборок нету. Выполняется второе условие однородности дисперсий.
Все три условия применимости дисперсионного анализа выполняются.


Проведём обычный однофакторный диспенрсионный анализ **one-way ANOVA**

In [ ]:
_,p_value=stats.f_oneway(football_players, hockey_players, weightlifters)
p_value

0.010482206918698694

Здесь p_value<alpha (0.01<0.05). Это значит между штангистами, футбалистами и хоккеистами статистически значимые различия в росте (и в среднем росте) обноружены. Данный тест не даёт конкретный ответ между какими парами групп существуют статистически значимые различия в среднем росте. Поэтому нужно провести **Post hoc test Тьюки** для трёх выборок.

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pandas as pd

Cоздадим лист меток групп, в котором указывается, к какой группе относится каждый элемент данных.

In [ ]:
groups = (['football_players']*len(football_players)+['hockey_players']*len(hockey_players)+['weightlifters']*len(weightlifters))

Объединим данные один массив

In [ ]:
common = np.concatenate([football_players, hockey_players, weightlifters])

Создадим DataFrame

In [ ]:
df = pd.DataFrame({'height': common, 'group': groups})

Выполним тест Тьюки

In [ ]:
tukey = pairwise_tukeyhsd(endog=df['height'], groups=df['group'], alpha=alpha)
print(f'{alpha=}')
print()
print(tukey)

alpha=0.05

          Multiple Comparison of Means - Tukey HSD, FWER=0.05          
     group1          group2     meandiff p-adj   lower    upper  reject
-----------------------------------------------------------------------
football_players hockey_players  -0.4583  0.979  -6.2732  5.3566  False
football_players  weightlifters  -6.3977 0.0219 -11.9583 -0.8372   True
  hockey_players  weightlifters  -5.9394 0.0284 -11.3181 -0.5607   True
-----------------------------------------------------------------------


**Проанализируем результат.**

Для этого смотрим на столбец p-adj (p_value) и сравниваем с уровнем статистической значимости alpha = 0.05. Т.е значения p-adj, которые меньше 0.05 говорят нам о наличии статистически значимых различий. Мы видим, что между  футболистами (football_players)   и штангистми (weightlifters) p-adj=0.0219<0.05, а также между хоккеистами (hockey_players) и штангистми (weightlifters) p-adj=0.0284<0.05 обнаружены статистически значимые различия и для этих строк reject = True. А вот между  футболистами (football_players) и хоккеистами (hockey_players) p-adj=0.972>0.05 не обнаружены статистически значимые различия и для этой строки reject = False.

Значит статистически значимые различия в росте (и в среднем росте) не обноружены только между футболистами и хоккеистами.  А вот между штангистами и футбалистами, а также между штангистами и хоккеистами статистически значимые различия в росте (и в среднем росте) обноружены.